In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

#資料讀進
df = pd.read_excel('Courses_T1 2019.xlsx')
df = df.drop(columns='Unnamed: 0')
df.head()

,Course Code,Title,Field,Level,Pre-requisites,Professor,ECTS,Language,Remarks,TIME
0,GESTS409,Advanced accounting,Accounting,Specialization,At least two courses in accounting,Jean-Paul LOOZEN,5.0,English,NaN,"[('Wednesday', '12:00 to 14:00'), ('Wednesday'..."
1,GESTS410,Advanced corporate finance,Finance,Intermediate,"Accounting, basic finance course",Kim OOSTERLINCK,5.0,English,NaN,"[('Wednesday', '16:00 to 18:00'), ('Wednesday'..."
2,GESTS402,Advanced finance,Finance,Intermediate,At least one class in finance,Laurent GHEERAERT,5.0,English,Description available on demand,"[('Tuesday', '10:00 to 12:00'), ('Tuesday', '1..."
3,GESTS403,Advanced Marketing,Marketing,Intermediate,Basic marketing class,Sandra ROTHENBERGER,5.0,English,NaN,"[('Monday', '10:00 to 12:00'), ('Monday', '10:..."
4,GESTS489,Advanced Marketing,Marketing,Intermediate,Basic knowledge in marketing,Sandra ROTHENBERGER,5.0,English,NaN,"[('Monday', '10:00 to 12:00'), ('Monday', '18:..."


In [2]:
#找出所有CourseCode
course_code = []
for code in df['Course Code']:
    course_code.append(code)

In [3]:
#設定網頁url與html
url = 'https://gehol.ulb.ac.be/gehol/Vue/HoraireCours.php?lang=en&semaine_unif=114'
res = requests.get(url)
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<!--[if lte IE 9]>
  <script src="//cdnjs.cloudflare.com/ajax/libs/html5shiv/3.6.2/html5shiv.js"></script>
  <script src="//cdnjs.cloudflare.com/ajax/libs/respond.js/1.3.0/respond.js"></script>
<!--[endif]--><!--[if lt IE 7]>
<html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="fr"><![endif]--><!--[if IE 7]>
<html class="no-js lt-ie9 lt-ie8" lang="fr"><![endif]--><!--[if IE 8]>
<html class="no-js lt-ie9" lang="fr"><![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="fr"><!--<![endif]-->
<head>
<!-- Encodage des caractères -->
<meta charset="utf-8"/>
<!-- Amélioration du rendu -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<!-- Description de la page -->
<meta content="Visualisez les horaires des cours donnés à l'Université libre de Bruxelles." name="description"/>
<meta content="mongehol, gehol, ulb, horaire, université libre, bruxelles, université libre de bruxelles" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Ty

In [4]:
#預設會使用到的空清單
Théorie_timetable = []

#跑出每堂課的url
for CODE in course_code:
    url = 'https://gehol.ulb.ac.be/gehol/Vue/HoraireCours.php?lang=en&semaine_unif=114'
    url = url + '&cours=' + CODE
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    
    #在loop中開始抓取課堂資訊
    course_info = []
    for info in soup.find_all('div', {'class':'tooltip_info'}):
        course_info.append(info.text)
        
    #課堂資訊中先整理資訊  
    time = []
    course_date = []
    for course in course_info:
        course = course.replace('\n', '')
        course = course.split(' ')
        
        #找正課時間
        if 'ThéorieLocation' in course:
            course_date.append(course[1])
            time.append(course[3:6])
            
    #整理正課時間清單
    course_time = []
    for i in time:
        course_time.append(' '.join(i))
    
    #合併正課日期與時間    
    Théorie_timetable.append(list(zip(course_date, course_time)))
    
print(Théorie_timetable)  

[[('Wednesday', '12:00 to 14:00'), ('Wednesday', '14:00 to 16:00'), ('Thursday', '12:00 to 14:00'), ('Friday', '10:00 to 12:00')], [('Wednesday', '16:00 to 18:00'), ('Wednesday', '16:00 to 18:00')], [('Tuesday', '10:00 to 12:00'), ('Tuesday', '12:00 to 14:00')], [('Monday', '10:00 to 12:00'), ('Monday', '10:00 to 12:00'), ('Monday', '18:00 to 20:00'), ('Tuesday', '12:00 to 14:00')], [('Monday', '10:00 to 12:00'), ('Monday', '18:00 to 20:00'), ('Tuesday', '10:00 to 12:00'), ('Thursday', '10:00 to 12:00'), ('Friday', '08:00 to 10:00')], [('Wednesday', '10:00 to 12:00'), ('Friday', '10:00 to 12:00'), ('Friday', '14:00 to 16:00'), ('Friday', '14:00 to 16:00'), ('Friday', '14:00 to 16:00')], [('Friday', '08:00 to 10:00'), ('Friday', '08:00 to 10:00'), ('Saturday', '09:00 to 12:00')], [('Wednesday', '08:00 to 10:00')], [('Tuesday', '08:00 to 10:00'), ('Tuesday', '10:00 to 12:00'), ('Tuesday', '13:00 to 17:00'), ('Tuesday', '14:00 to 16:00'), ('Tuesday', '14:00 to 16:00'), ('Friday', '14:00 t

In [16]:
#遺漏資訊測試
missed_courses = []
for i in range(len(Théorie_timetable)):
    if Théorie_timetable[i] == []:
        temp_code = df[df.index == i]['Course Code'].values[0]
        missed_courses.append(temp_code)
missed_courses

[]

In [52]:
#建立上課時間欄位並儲存 (不使用dictionary是因為有些日期會重複)
df['TIME'] = Théorie_timetable
df.to_excel('Courses_T1 2019.xlsx')